In [27]:
import pandas as pd
from bson import ObjectId
import numpy as np
import glob
import os
import requests

In [5]:
!ls datasets/kf

all.txt		      read_14.txt	    write_attach_22.txt
edit_10.txt	      read_15.txt	    write_attach_23.txt
edit_1.txt	      read_16.txt	    write_attach_24.txt
edit_2.txt	      read_17.txt	    write_attach_25.txt
edit_3.txt	      read_18.txt	    write_attach_26.txt
edit_4.txt	      read_19.txt	    write_attach_27.txt
edit_5.txt	      read_1.txt	    write_attach_28.txt
edit_6.txt	      read_20.txt	    write_attach_29.txt
edit_7.txt	      read_2.txt	    write_attach_30.txt
edit_8.txt	      read_3.txt	    write_attach_scaffold_31.txt
edit_9.txt	      read_4.txt	    write_attach_scaffold_32.txt
edit_scaffold_10.txt  read_5.txt	    write_attach_scaffold_33.txt
edit_scaffold_11.txt  read_6.txt	    write_attach_scaffold_34.txt
edit_scaffold_1.txt   read_7.txt	    write_attach_scaffold_35.txt
edit_scaffold_2.txt   read_8.txt	    write_attach_scaffold_36.txt
edit_scaffold_3.txt   read_9.txt	    write_attach_scaffold_37.txt
edit_scaffold_4.txt   remove_whitespace.py  write_attach_scaffold_38.txt
e

In [32]:
type_to_cat = {'GET': [1.0,0.0,0.0,0.0], 'POST': [0.0,1.0,0.0,0.0], 'PUT': [0.0,0.0,1.0,0.0], 'DELETE': [0.0,0.0,0.0,1.0]}
obj_type_to_id = {'': 0, 'Note': 1, 'Scaffold': 2, 'Attachment': 3, 'Context': 4}
init_row = pd.DataFrame({'type': 'GET', 'url': 'start', 'time': 'time', 'ip': '127.0.0.1', 'delta': 0.0, 'seq_type': 'start'},index =[0])
end_row = pd.DataFrame({'type': 'GET', 'url': 'end', 'time': 'time', 'ip': '127.0.0.1', 'delta': 0.0, 'seq_type': 'end'},index =[0])

In [7]:
urls = ['start',
        '/api/notifications/notify/objectId',
        '/api/communities/objectId/groups',
        '/api/links/from/objectId',
        '/api/links/objectId/search',
        '/api/links/either/objectId',
        '/api/records/object/objectId',
        '/api/records/read/objectId/objectId',
        '/api/links',
        '/api/contributions/objectId',
        '/api/links/',
        '/api/upload',
        '/api/objects/objectId',
        '/api/communities/objectId',
        '/api/authors/objectId/me',
        '/api/links/objectId',
        '/api/objects/objectId/objectId',
        '/api/links/to/objectId',
       'end']

In [19]:
seq_types = [
    'start',
    'get_note',
    'get_links_from',
    'record_note_read',
    'get_links_from_contrib',
    'edit_note',
    'notify_comm',
    'record_note_edited',
    'get_object',
    'get_groups',
    'get_links_to_note',
    'search',
    'post_scaffold',
    'get_community',
    'get_note_records',
    'get_author',
    'delete_scaffold',
    'new_note',
    'post_link_view_note',
    'new_attachment',
    'upload_attachment',
    'edit_object',
    'post_link',
    'end'
]

In [8]:
filename = "./datasets/kf/edit_1.txt"

In [9]:
seq = pd.read_csv(filename, names=['type', 'url', 'time', 'ip', 'delta', 'seq_type'])

In [10]:
seq

,type,url,time,ip,delta,seq_type
0,GET,/api/objects/60624940620841af71f6a3b2,22:47:08,127.0.0.1,0.0,get_note
1,GET,/api/links/from/5f03b3bd13c79557224b7365,22:47:09,127.0.0.1,1.0,get_links_from
2,POST,/api/records/read/5f03b3bd13c79557224b7363/606...,22:47:09,127.0.0.1,0.0,record_note_read
3,GET,/api/links/from/60624940620841af71f6a3b2,22:47:22,127.0.0.1,13.0,get_links_from_contrib
4,PUT,/api/objects/5f03b3bd13c79557224b7363/60624940...,22:47:22,127.0.0.1,0.0,edit_note
5,POST,/api/notifications/notify/5f03b3bd13c79557224b...,22:47:22,127.0.0.1,0.0,notify_comm
6,POST,/api/records/read/5f03b3bd13c79557224b7363/606...,22:47:22,127.0.0.1,0.0,record_note_edited


In [11]:
def normalize_url(url):
    parts = url.split("/")
    parts = [part if (not ObjectId.is_valid(part)) else 'objectId' for part in parts]
    return "/".join(parts)

In [29]:
def getObjectType(normalized_url, url):
    urls = [
        '/api/links/from/objectId',
        '/api/links/either/objectId',
        '/api/objects/objectId',
        '/api/objects/objectId/objectId',
        '/api/links/to/objectId'
    ]
    if normalized_url in urls: #Find object type
        objectId = url.split("/")[-1]
        r = requests.get(f"http://localhost:9000/api/objects/type/{objectId}")
        try:
            return r.json()['type']
        except:
            print ("***********Error ", url, normalized_url)
            print(r)
            return ''
    if normalized_url == '/api/contributions/objectId':
        pass #this is a post request, need info from post data
    if normalized_url == '/api/links/objectId':
        objectId = url.split("/")[-1]
        r = requests.get(f"http://localhost:9000/api/links/type/{objectId}")
        try:
            return r.json()['type']
        except:
            print ("***********Error ", url)
            print(r)
            return ''
    return ''

In [12]:
def add_type_cat(seq):
    types = np.array([type_to_cat[t] for t in seq.type])
    return pd.concat([seq, pd.DataFrame(types, columns=['t_0', 't_1', 't_2', 't_3'])], axis=1)

In [13]:
def add_url_cat(seq):
    one_hot_urls = []
    for url in seq.url:
        url = url.strip()
        n_url = normalize_url(url)
        one_hot_url = np.zeros(len(urls))
        one_hot_url[urls.index(n_url)] = 1
        one_hot_urls.append(one_hot_url)
    return pd.concat([seq, 
                      pd.DataFrame(np.array(one_hot_urls),
                                   columns=[f"url_{i}" for i in range(len(urls))])], axis=1)

In [16]:
def add_init_end_action(seq):
    return pd.concat([init_row, seq, end_row]).reset_index(drop = True)

In [20]:
def add_seq_type_cat(seq):
    one_hot_seq_types = []
    for seq_type in seq.seq_type:
        seq_type = seq_type.strip()
        one_hot_seq_type = np.zeros(len(seq_types))
        one_hot_seq_type[seq_types.index(seq_type)] = 1
        one_hot_seq_types.append(one_hot_seq_type)
    return pd.concat([seq, pd.DataFrame(np.array(one_hot_seq_types), 
                                        columns=[f"seq_type_{i}" for i in range(len(seq_types))])], axis=1)

In [38]:
def add_obj_type(df):
    types = []
    type_ids = []
    for url in df.url:
        norm_url = normalize_url(url)
        objType = getObjectType(norm_url, url)
        types.append(objType)
        type_ids.append(obj_type_to_id[objType])
        print(f"url:{url} type:{objType}")
    df['objType'] = types
    df['objTypeId'] = type_ids

In [39]:
def process_seq(seq):
    seq = add_init_end_action(seq)
    add_obj_type(seq)
    seq = add_type_cat(seq)
    seq = add_url_cat(seq)
    seq = add_seq_type_cat(seq)
    return seq

In [41]:
p_seq = process_seq(seq)

url:start type:
url:/api/objects/60624940620841af71f6a3b2 type:Note
url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/records/read/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:
url:/api/links/from/60624940620841af71f6a3b2 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:
url:end type:


In [42]:
p_seq.columns

Index(['type', 'url', 'time', 'ip', 'delta', 'seq_type', 'objType',
       'objTypeId', 't_0', 't_1', 't_2', 't_3', 'url_0', 'url_1', 'url_2',
       'url_3', 'url_4', 'url_5', 'url_6', 'url_7', 'url_8', 'url_9', 'url_10',
       'url_11', 'url_12', 'url_13', 'url_14', 'url_15', 'url_16', 'url_17',
       'url_18', 'seq_type_0', 'seq_type_1', 'seq_type_2', 'seq_type_3',
       'seq_type_4', 'seq_type_5', 'seq_type_6', 'seq_type_7', 'seq_type_8',
       'seq_type_9', 'seq_type_10', 'seq_type_11', 'seq_type_12',
       'seq_type_13', 'seq_type_14', 'seq_type_15', 'seq_type_16',
       'seq_type_17', 'seq_type_18', 'seq_type_19', 'seq_type_20',
       'seq_type_21', 'seq_type_22', 'seq_type_23'],
      dtype='object')

In [43]:
seq_filenames = glob.glob("./datasets/kf/*txt")

In [45]:
for filename in seq_filenames:
    seq = pd.read_csv(filename,names=['type', 'url', 'time', 'ip', 'delta', 'seq_type'])
    seq = process_seq(seq)
    new_filename = os.path.basename(filename).split(".")[0]
    seq.to_csv(f"./datasets/kf/processed2/{new_filename}.csv", index=False)

url:start type:
url:/api/objects/60624eaa620841af71f6a4a2 type:Note
url:/api/communities/5f03b3bd13c79557224b7363 type:
url:/api/links/from/5f03b3be13c79557224b7368 type:Scaffold
url:/api/records/read/5f03b3bd13c79557224b7363/60624eaa620841af71f6a4a2 type:
***********Error  /api/links/60624eda620841af71f6a4a8
<Response [404]>
url:/api/links/60624eda620841af71f6a4a8 type:
***********Error  /api/links/60624eda620841af71f6a4aa
<Response [404]>
url:/api/links/60624eda620841af71f6a4aa type:
***********Error  /api/links/60624eda620841af71f6a4ac
<Response [404]>
url:/api/links/60624eda620841af71f6a4ac type:
url:/api/links type:
***********Error  /api/links/60624eda620841af71f6a4a7
<Response [404]>
url:/api/links/60624eda620841af71f6a4a7 type:
url:/api/links type:
url:/api/objects/5f03b3bd13c79557224b7363/60624eaa620841af71f6a4a2 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624eaa620841af71f6a4a2 type:
url:/api/links/5f

url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/records/read/5f03b3bd13c79557224b7363/60624972620841af71f6a3bd type:
url:/api/links/from/60624972620841af71f6a3bd type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60624972620841af71f6a3bd type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624972620841af71f6a3bd type:
url:/api/objects/60624add620841af71f6a404 type:Note
url:/api/links/to/60624add620841af71f6a404 type:Note
url:/api/objects/5f03b3be13c79557224b7368 type:Scaffold
url:/api/records/read/5f03b3bd13c79557224b7363/60624add620841af71f6a404 type:
url:/api/links/from/60624add620841af71f6a404 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60624add620841af71f6a404 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624add620841af71f6a404 type:
url:/api/links/5f03b3bd13c79557224b7363/search type:
url:/api/objects/60624a4662084

url:/api/links/from/5f5bb49a28716a106cce10de type:Scaffold
url:/api/records/read/5f243b4d4fe00c7291dd4f4a/5f69971133f00b8176e11dfb type:
url:/api/records/object/5f69971133f00b8176e11dfb type:
url:/api/communities/5f243b4d4fe00c7291dd4f4a/groups type:
url:/api/objects/5f7615d80d1110fb916446bd type:Note
url:/api/links/from/5f7615d80d1110fb916446bd type:Note
url:/api/records/read/5f243b4d4fe00c7291dd4f4a/5f7615d80d1110fb916446bd type:
url:/api/objects/5f6aa0a966a9d2b444898ef4 type:Note
url:/api/links/from/5f243b4e4fe00c7291dd4f4f type:Scaffold
url:/api/objects/5f6264e6016244f03974f9ef type:Scaffold
url:/api/records/read/5f243b4d4fe00c7291dd4f4a/5f6aa0a966a9d2b444898ef4 type:
url:/api/objects/5f6115847ba0c5aa0136094e type:Note
url:/api/links/from/5f6115847ba0c5aa0136094e type:Note
url:/api/links/from/5f243b4d4fe00c7291dd4f4c type:Context
url:/api/records/read/5f243b4d4fe00c7291dd4f4a/5f6115847ba0c5aa0136094e type:
url:/api/objects/5f67d7e778104f4efbf47a7e type:Note
url:/api/links/from/5f67

url:/api/objects/5f03b3bd13c79557224b7363/60633c2d620841af71f6a59f type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60633c2d620841af71f6a59f type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/links/ type:
url:/api/objects/5f03b3be13c79557224b7368 type:Scaffold
url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/60633c7a620841af71f6a5bb type:Attachment
url:/api/links type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/60633c7d620841af71f6a5c2 type:Attachment
url:/api/links type:
url:/api/links/to/60633c6f620841af71f6a5b6 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60633c6f620841af71f6a5b6 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7

url:/api/objects/60624940620841af71f6a3b2 type:Note
url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/records/read/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:
url:/api/links/from/60624940620841af71f6a3b2 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624940620841af71f6a3b2 type:
url:end type:
url:start type:
url:/api/objects/606249f9620841af71f6a3d3 type:Note
url:/api/links/from/5f03b3be13c79557224b7368 type:Scaffold
url:/api/records/read/5f03b3bd13c79557224b7363/606249f9620841af71f6a3d3 type:
url:/api/links/from/606249f9620841af71f6a3d3 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/606249f9620841af71f6a3d3 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/606249f9620841af71f6a3d3 type:
url:end type:
url:start type:
url:/api/objects/6062

url:/api/objects/5f873e94ae45facb50bddfdc type:Scaffold
url:/api/records/read/5f03b3bd13c79557224b7363/60624c1c620841af71f6a435 type:
***********Error  /api/links/60624c57620841af71f6a43b
<Response [404]>
url:/api/links/60624c57620841af71f6a43b type:
***********Error  /api/links/60624c57620841af71f6a43a
<Response [404]>
url:/api/links/60624c57620841af71f6a43a type:
url:/api/objects/5f03b3bd13c79557224b7363/60624c1c620841af71f6a435 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624c1c620841af71f6a435 type:
url:/api/links/5f03b3bd13c79557224b7363/search type:
url:end type:
url:start type:
url:/api/objects/60624875620841af71f6a391 type:Note
url:/api/objects/5f03b3be13c79557224b7368 type:Scaffold
url:/api/records/read/5f03b3bd13c79557224b7363/60624875620841af71f6a391 type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/objects/5f03

url:start type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/objects/60633a3f620841af71f6a527 type:Note
url:/api/links/ type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/60633a50620841af71f6a52c type:Attachment
url:/api/links type:
url:/api/links/to/60633a3f620841af71f6a527 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60633a3f620841af71f6a527 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60633a3f620841af71f6a527 type:
url:end type:
url:start type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/links/ type:
url:/api/links/from/60633a91620841af71f6a53b type:Note
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/60633ab0620841af71f6a540 type:Attachment
url:/api/links type:
url:/api/links/from/60633a91620841af71f6a53b type:Note
url:/api/ob

url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/60634214620841af71f6a6bf type:Attachment
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/links type:
url:/api/objects/5f03b3bd13c79557224b7363/606341fc620841af71f6a6ba type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/links/5f03b3bd13c79557224b7363/search type:
url:/api/records/read/5f03b3bd13c79557224b7363/606341fc620841af71f6a6ba type:
url:end type:
url:start type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/links/ type:
url:/api/objects/5f03b3be13c79557224b7368 type:Scaffold
url:/api/links/from/5f03b3be13c79557224b7368 type:Scaffold
url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/upload type:
url:/api/objects/5f03b3bd13c79557224b7363/606342636208

***********Error  /api/links/60624e68620841af71f6a497
<Response [404]>
url:/api/links/60624e68620841af71f6a497 type:
url:/api/objects/5f03b3bd13c79557224b7363/60624e46620841af71f6a491 type:Note
url:/api/notifications/notify/5f03b3bd13c79557224b7363 type:
url:/api/records/read/5f03b3bd13c79557224b7363/60624e46620841af71f6a491 type:
url:end type:
url:start type:
url:/api/objects/5f6aa0a966a9d2b444898ef4 type:Note
url:/api/links/from/5f243b4e4fe00c7291dd4f4f type:Scaffold
url:/api/objects/5f6264e6016244f03974f9ef type:Scaffold
url:/api/records/read/5f243b4d4fe00c7291dd4f4a/5f6aa0a966a9d2b444898ef4 type:
url:end type:
url:start type:
url:/api/contributions/5f03b3bd13c79557224b7363 type:
url:/api/links/ type:
url:/api/objects/5f62b7c64f1b95edacd7c788 type:Scaffold
url:/api/links/from/5f03b3bd13c79557224b7365 type:Context
url:/api/links/to/60624a46620841af71f6a3e9 type:Note
url:/api/objects/5f03b3bd13c79557224b7363/60624a46620841af71f6a3e9 type:Note
url:/api/notifications/notify/5f03b3bd13c7